In [ ]:
import os

import tensorflow as tf
tf.config.experimental_run_functions_eagerly(True)

import h5py
import numpy as np

tf.test.gpu_device_name()

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

# Loading datasets

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import preprocess_input

In [ ]:
def load_dataset_h5(path, dataset_name):
    with h5py.File(path, "r") as hf:
        print(hf.keys())
        X = hf[dataset_name][:]
        hf.close()
    return X

In [ ]:
path = '/content/gdrive/MyDrive/masterDB/train.h5'
path_val = '/content/gdrive/MyDrive/val.h5'

In [ ]:
X_train = load_dataset_h5(path, 'X_train')
y_train = load_dataset_h5(path, 'y_train').reshape((-1,1))

In [ ]:
X_val = load_dataset_h5(path_val, 'X_val')[:1500]
y_val = load_dataset_h5(path_val, 'y_val')[:1500].reshape((-1,1))

In [ ]:
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.utils import to_categorical


In [ ]:
X_train= preprocess_input(X_train)
X_val = preprocess_input(X_val)

# Create model

In [ ]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, AveragePooling2D

In [ ]:
def get_xception()
    base_model = Xception(include_top=False, weights=None, classes=2)
    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

In [ ]:
# results = model.fit(train_dataset, validation_data=test_dataset, epochs=10)
results = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)

## Evaluation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
saving_dir = "/content/gdrive/MyDrive/fake-faces-detector/exp/models/Xception"

In [ ]:
losses = pd.DataFrame(model.history.history)
losses.to_csv(os.path.join(saving_dir, 'losses.csv'))

In [ ]:
losses[['loss', 'val_loss']].plot()
plt.savefig(os.path.join(saving_dir, 'loss.png'))

In [ ]:
losses[['accuracy', 'val_accuracy']].plot()
plt.savefig(os.path.join(saving_dir, 'acc.png'))

In [ ]:
model.save(os.path.join(saving_dir, 'model.h5'))